### Колёсная одометрия

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
calibration_localization = pd.read_csv("calibration_localization.csv").sort_values(by='stamp')
calibration_odometry = pd.read_csv("calibration_odometry.csv").sort_values(by='stamp')
localization = pd.read_csv("localization.csv").sort_values(by='stamp')
odometry = pd.read_csv("odometry.csv").sort_values(by='stamp')

In [2]:
calibration_odometry[:2]

,stamp,steering_is_set,steering,wheel_velocities_is_set,front_left_velocity,front_right_velocity,rear_left_velocity,rear_right_velocity
0,1.518461e+09,False,0.0,True,5.0,4.980556,4.913889,4.919444
1,1.518461e+09,True,-2.0,False,0.0,0.000000,0.000000,0.000000


In [3]:
calibration_odometry[:2]

,stamp,steering_is_set,steering,wheel_velocities_is_set,front_left_velocity,front_right_velocity,rear_left_velocity,rear_right_velocity
0,1.518461e+09,False,0.0,True,5.0,4.980556,4.913889,4.919444
1,1.518461e+09,True,-2.0,False,0.0,0.000000,0.000000,0.000000


#### Параметры машины:
- steering_base - сдвиг начального значения поворота руля
- front_wheel_angle_to_steering - коэффициент пропорциональности между углом поворота передних колёс и углом поворота руля
- rear_left_coefficient - линейная поправка скорости заднего левого колеса
- rear_right_coefficient - линейная поправка скорости заднего правого колеса

In [4]:
steering_base = 5.0
front_wheel_angle_to_steering = 12.7
rear_right_coefficient = 1.0086226141262378
rear_left_coefficient = 1.009079525155482
rear_track_width = 1.65

### Пример работы одометрии на задних колёсах:

In [5]:
import math
import time

def get_linear_velocity(rear_right_velocity, rear_left_velocity):
    #Написать функция

def get_angular_velocity(rear_right_velocity, rear_left_velocity, rear_track_width):
    #Написать функцию

def update_position(position, dt, linear_velocity, angular_velocity):
    if (abs(angular_velocity) > 1e-6):
        phi = angular_velocity * dt
        # cuvrature radius
        radius = linear_velocity / angular_velocity
        movement = [radius * math.sin(phi), radius * (1 - math.cos(phi))]
        position['yaw'] += phi
    else:
        #Написать этот вариант
    yaw = position['yaw']
    #Тут должно быть что-то типо position += повернуть movement на yaw
    position['stamp'] += dt

def get_odom_localization(init_position, odometry, rear_track_width):
    odom_x = []
    odom_y = []
    break_flag = False
    processed_total = 0
    last_time_published = time.time()
    rear_wheel_odometry = odometry[odometry['wheel_velocities_is_set'] == True]
    total = float(len(rear_wheel_odometry['stamp']))
    last_rear_right_velocity = 0.0
    last_rear_left_velocity = 0.0
    for odometry_idx in rear_wheel_odometry.T:
        odom_stamp = rear_wheel_odometry['stamp'][odometry_idx]
        odom_x.append(position['x'])
        odom_y.append(position['y'])
        rear_right_velocity = rear_wheel_odometry['rear_right_velocity'][odometry_idx]
        rear_left_velocity = rear_wheel_odometry['rear_left_velocity'][odometry_idx]
        rear_right_velocity *= rear_right_coefficient
        rear_left_velocity *= rear_left_coefficient
        linear_velocity = get_linear_velocity(last_rear_right_velocity, last_rear_left_velocity)
        angular_velocity = get_angular_velocity(last_rear_right_velocity, last_rear_left_velocity, rear_track_width)

        dt = odom_stamp - position['stamp']
        update_position(position, dt, linear_velocity, angular_velocity)
        processed_total += 1
        if (last_time_published + 1.0 < time.time()):
            last_time_published = time.time()
            print 'Processed = ', (processed_total / total)
        last_rear_right_velocity = rear_right_velocity
        last_rear_left_velocity = rear_left_velocity
    print 'Processed = ', (processed_total / total)
    return odom_x, odom_y

position = {'x' : calibration_localization['x'][0],
            'y' : calibration_localization['y'][0],
            'yaw' : calibration_localization['yaw'][0],
            'stamp' : calibration_localization['stamp'][0]}
odom_x, odom_y = get_odom_localization(position, calibration_odometry, rear_track_width)

IndentationError: expected an indented block (<ipython-input-5-d17435c0864d>, line 7)

### Визуализация результатов работы

In [6]:
def show_locs(loc_x, loc_y, odom_x, odom_y):
    fig = plt.figure()
    plt.plot(loc_x, loc_y)
    plt.plot(odom_x, odom_y)
    plt.ylabel('y')
    plt.xlabel('x')
    plt.grid(True)
    plt.show()
show_locs(calibration_localization['x'], calibration_localization['y'], odom_x, odom_y)

NameError: name 'odom_x' is not defined

### Набросать функцию, результат которой мы будет минимизировать

In [9]:
import copy
def diff(rear_track_width):
    max_loc_idx = calibration_localization.shape[0] - 1
    loc_idx = 1
    result_diff = []
    position = copy.deepcopy(calibration_localization.T[1])
    odom_position = copy.deepcopy(calibration_localization.T[0])
    #Вот тут должна быть copy-past-а из get_odom_localization с доработками
    return np.array(result_diff)

In [10]:
def diff_params(params):
    res = diff(params[0])
    print 'Step result =', sum(res ** 2), 'params=', params
    return res
from scipy.optimize import least_squares
res = least_squares(fun=diff_params, x0=[ 1.5], bounds=([1.45], [1.65]), diff_step=[0.1], max_nfev=5)

Step result = 0 params= [ 1.5]
Step result = 0 params= [ 1.65]


In [11]:
diff_params([1.5])
diff_params([1.54])
diff_params([1.55])

Step result = 0 params= [1.5]
Step result = 0 params= [1.54]
Step result = 0 params= [1.55]


array([], dtype=float64)

In [ ]:
best_rear_track_width = ....

### Результат после подбора параметров

In [12]:
import math
import time
position = {'x' : calibration_localization['x'][0],
            'y' : calibration_localization['y'][0],
            'yaw' : calibration_localization['yaw'][0],
            'stamp' : calibration_localization['stamp'][0]}
odom_x, odom_y = get_odom_localization(position, calibration_odometry, best_rear_track_width)
#show_locs(calibration_localization['x'], calibration_localization['y'], odom_x, odom_y)

NameError: name 'get_odom_localization' is not defined

#### Посмотрим - на все ли проезды наш параметр идеален:

In [13]:
position = {'x' : localization['x'][0],
            'y' : localization['y'][0],
            'yaw' : localization['yaw'][0],
            'stamp' : localization['stamp'][0]}
odom_x, odom_y = get_odom_localization(position, odometry, rear_track_width)
#show_locs(localization['x'], localization['y'], odom_x, odom_y)

NameError: name 'get_odom_localization' is not defined

In [14]:
position = {'x' : localization['x'][0],
            'y' : localization['y'][0],
            'yaw' : localization['yaw'][0],
            'stamp' : localization['stamp'][0]}
odom_x, odom_y = get_odom_localization(position, odometry, best_rear_track_width)
#show_locs(localization['x'], localization['y'], odom_x, odom_y)

NameError: name 'get_odom_localization' is not defined

### Почему последнии графики такие такие они есть?